In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
from time import time
from tqdm import tqdm

import numpy as np
import pandas as pd
import scipy.io as sio
import matplotlib.pyplot as plt

import tensorflow as tf
from pix2vid2 import make_model
from utils import check_tf_gpu, describe_data, load_data

NUM_REALIZATIONS = 1000
NX,  NY,  NZ = 64, 64, 1
NTT, NT1, NT2 = 40, 20, 5
HIDDEN = [16, 64, 128]

folder     = 'SlopingAquiferSmall/simulationsVE'
sec2year   = 365.25 * 24 * 60 * 60
Darcy      = 9.869233e-13
psi2pascal = 6894.76
co2_rho    = 686.5266
milli      = 1e-3
mega       = 1e6
check_tf_gpu()

In [ ]:
gt = sio.loadmat('{}/grids/Gt.mat'.format(folder), simplify_cells=True)['Gt']
gtops = gt['cells']['z'].reshape(NX,NY,order='F')
gvols = gt['parent']['cells']['volumes'].reshape(NX,NY,order='F')
ghght = gt['cells']['H'].reshape(NX,NY,order='F')

deltatime = sio.loadmat('{}/data/time_arr.mat'.format(folder), simplify_cells=True)['time_arr']
timesteps = np.cumsum(deltatime)
timesteps_inj = timesteps[:20]
timesteps_mon = timesteps[20:][[0, 4, 9, 7]] #timesteps[[19, 24, 29, 34, 39]]

print('timesteps: {} | deltatime: {}'.format(len(timesteps), np.unique(deltatime)))
print('injection: {}'.format(timesteps_inj))
print('monitoring: {}'.format(timesteps_mon))

In [ ]:
datas, norms = load_data(folder)
X_data, c_data, y1_data, y2_data = datas.values()

In [ ]:
train_idx = np.load('{}/models/training_idx.npy'.format(folder))
test_idx  = np.setdiff1d(range(len(X_data)), train_idx)

tridx = train_idx[:20]
teidx = test_idx[:20]

X_train  = tf.cast(X_data[tridx], tf.float32)
c_train  = tf.cast(c_data[tridx], tf.float32)
y1_train = tf.cast(y1_data[tridx], tf.float32)
y2_train = tf.cast(y2_data[tridx], tf.float32)

X_test  = tf.cast(X_data[teidx], tf.float32)
c_test  = tf.cast(c_data[teidx], tf.float32)
y1_test = tf.cast(y1_data[teidx], tf.float32)
y2_test = tf.cast(y2_data[teidx], tf.float32)

In [ ]:
model = make_model(hidden=HIDDEN)
model.load_weights('{}/models/pix2vid-v2.weights.h5'.format(folder))

In [ ]:
y1_train_pred, y2_train_pred = model.predict([X_train, c_train], verbose=False)
y1_test_pred, y2_test_pred = model.predict([X_test, c_test], verbose=False)

y1_train_pred = np.array(y1_train_pred)
y2_train_pred = np.array(y2_train_pred)[:,[]]
y1_test_pred = np.array(y1_test_pred)
y2_test_pred = np.array(y2_test_pred)

X_train = np.array(X_train)
c_train = np.array(c_train)
y1_train = np.array(y1_train)
y2_train = np.array(y2_train)

X_test = np.array(X_test)
c_test = np.array(c_test)
y1_test = np.array(y1_test)
y2_test = np.array(y2_test)

print('Train - y1: {} | y2: {}'.format(y1_train_pred.shape, y2_train_pred.shape))
print('Test - y1: {} | y2: {}'.format(y1_test_pred.shape, y2_test_pred.shape))

In [ ]:
hues = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple']
plt.figure(figsize=(15,7.5))
for i in range(15):
    plt.subplot(3, 5, i+1)
    d = c_data[i]
    nw = len(np.unique(d.sum(0)))-1
    for w in range(nw):
        for t in range(len(timesteps_inj)-1):
            plt.hlines(d[t,w], timesteps_inj[t], timesteps_inj[t+1], color=hues[w])
            plt.vlines(timesteps_inj[t+1], d[t,w], d[t+1,w], color=hues[w], ls=':', lw=0.5)
    plt.title('R{} | ({})'.format(i, nw))
    plt.xlim(0.5,10)
    plt.ylim(0,None)
    plt.xticks(np.arange(1,11))
    plt.grid(True, which='both', alpha=0.25)
    plt.xlabel('Time (years)') if i > 9 else None
    plt.ylabel('Inj. Rate (MT CO2)') if i % 5 == 0 else None
plt.tight_layout()
plt.show()

***
### Optimization

In [ ]:
porevol = np.load('{}/simulations/data/porevol.npy'.format(folder))
porevol.shape

In [ ]:
sample = 123

dc = c_train[sample].sum()
print(dc)

dd = (y2_train[sample, -1].squeeze() * porevol[sample]) / 1e3 / mega * co2_rho
print(np.sum(dd))

plt.imshow(dd, cmap='jet')
plt.colorbar()
plt.show()